In [13]:
import librosa
import numpy as np
import os
import math
import pickle
from sklearn.cluster import KMeans
import hmmlearn.hmm
from collections import defaultdict

result = defaultdict(list)

In [14]:
def get_mfcc(file_path):
  y, sr = librosa.load(file_path) # read .wav file
  hop_length = math.floor(sr*0.010) # 10ms hop
  win_length = math.floor(sr*0.025) # 25ms frame
  # mfcc is 12 x T matrix
  mfcc = librosa.feature.mfcc(
      y, sr, n_mfcc=12, n_fft=1024,
      hop_length=hop_length, win_length=win_length)
  # substract mean from mfcc --> normalize mfcc
  mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
  # delta feature 1st order and 2nd order
  delta1 = librosa.feature.delta(mfcc, order=1)
  delta2 = librosa.feature.delta(mfcc, order=2)
  # X is 36 x T
  X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
  # return T x 36 (transpose of X)
  return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
  files = os.listdir(data_dir)
  mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
  return mfcc

def clustering(X, n_clusters=20):
  kmeans = KMeans(n_clusters=n_clusters, n_init=100, random_state=0, verbose=0)
  kmeans.fit(X)
  print("centers", kmeans.cluster_centers_.shape)
  return kmeans  

In [15]:
class_names = ["toi", "nguoi", "dich",  "theo", "benh_nhan"]
dataset = {}
train_dataset = {}
for cname in class_names:
  dataset[cname] = get_class_data(os.path.join("train", cname))
  print(f"Loaded {cname} dataset - ", len(dataset[cname]))
  dataset[cname] = dataset[cname][:-20]

  # Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in dataset.items()], axis=0)
print("vectors", all_train_vectors.shape)
  # Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)
print("Clustering DONE")

/home/apple/anaconda3/envs/voice/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


Loaded toi dataset -  113
Loaded nguoi dataset -  134
Loaded dich dataset -  127
Loaded theo dataset -  116
Loaded benh_nhan dataset -  196
vectors (25723, 36)
centers (20, 36)
centers (20, 36)
Clustering DONE


In [19]:
dict_components = {
    # toi |t|~|o|~|i|
    "toi": 9,
    # dich |d|~|i|~|ch|
    "dich": 9,
    # nguoi |ng|~|uo|~|i|
    "nguoi": 9,
    # theo |th|~|e|~|o|
    "theo": 9,
    # benh_nhan |b|~|e|~|nh|~|silent|~|nh|~|a|~|n| 
    "benh_nhan": 18,
}

In [28]:
models = {}
for cname in class_names:
  print("====", cname)
    # convert all vectors to the cluster index
    # dataset['cname'] = [O^1, ... O^R]
    # O^r = (c1, c2, ... ct, ... cT)
    # O^r size T x 1
  train_dataset[cname] = list([kmeans.predict(v).reshape(-1,1) for v in dataset[cname]])
    
  # Initialize Model
#   n = dict_components[cname]
#   startprob = np.zeros(n)
#   startprob[0] = 1.0
#   transmat = np.diag(np.full(n,1))

#   hmm = hmmlearn.hmm.GMMHMM(
#     n_components=n, 
#     n_mix = 4, random_state=10, n_iter=500, verbose=True,
#     params='mctw', init_params='mct',
#     startprob_prior=startprob,
#     transmat_prior=transmat,
#   )


  if cname == "toi":
#     # =================================================================
#     # toi |sil|~|t|~|o|~|i|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=11, random_state=10, n_iter=500, init_params='e', params='ste', verbose=True
    ) 
    hmm.startprob_ = np.array([0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])
#     hmm.startprob_ = np.zeros(11)
#     hmm.startprob_[0] = 1.0
#     hmm.transmat_ = np.diag(np.full(11, 1))
#     print(hmm.transmat_)

#     hmm = hmmlearn.hmm.MultinomialHMM(
#       n_components=9, init_params='e', params='ste', verbose=True
#     ) 
#     hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
#     hmm.transmat_ = np.array([
#         [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
#       ])

  elif cname == "nguoi":
    # =================================================================
    # nguoi |sil|~|ng|~|u|~o|~|i|~|sil|
    
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=17, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
    ])

  elif cname == "dich":
    # =================================================================
    # dich |sil|~|d|~|i|~|c|~h|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=14, random_state=10, n_iter=500, init_params='e', params='ste', verbose=True
    )
    hmm.startprob_ = np.array([0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])


  elif cname == "theo":
    # =================================================================
    # theo |sil|~|th|~|e|~|o|~|sil|
    hmm = hmmlearn.hmm.MultinomialHMM(
      n_components=11, random_state=10, n_iter=500, init_params='e', params='ste', verbose=True
    )
#     hmm.startprob_ = np.zeros(11)
#     hmm.startprob_[0] = 1.0
#     hmm.transmat_ = np.diag(np.full(11, 1))
    hmm.startprob_ = np.array([0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
      ])


  elif cname == "benh_nhan":
    # =================================================================
    # benh_nhan |sil|~|b|~|e|~|nh|~|silent|~|nh|~|a|~|n|~|sil|  
    hmm = hmmlearn.hmm.MultinomialHMM(
        n_components=21, random_state=10, n_iter=500, init_params='e', params='ste', verbose=True
    )
#     hmm.startprob_ = np.zeros(21)
#     hmm.startprob_[0] = 1.0
#     hmm.transmat_ = np.diag(np.full(21, 1))
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
    hmm.transmat_ = np.array([
        [0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],    
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.3, 0.2],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
    ])

#   if cname[:4] != "test":
  X = np.concatenate(train_dataset[cname])
#   lengths = list([len(x) for x in train_dataset[cname]])
#       print("training class", cname)
#       print(X.shape, lengths, len(lengths))
  hmm.fit(X)
#       print(hmm.transmat_)
  models[cname] = hmm
  with open(os.path.join("models", cname + ".pkl"), "wb") as file: 
    pickle.dump(models[cname], file)
    print("Training done. Model has been dump to ", os.path.join("models", cname + ".pkl"))
print("Training Done")

==== toi


         1      -12723.5455             +nan
         2       -9125.1904       +3598.3551
         3       -9097.9092         +27.2813
         4       -9094.4643          +3.4448
         5       -9093.1675          +1.2968
         6       -9092.2399          +0.9276
         7       -9091.3848          +0.8550
         8       -9090.7238          +0.6611
         9       -9090.4195          +0.3043
        10       -9090.3019          +0.1176
        11       -9090.2470          +0.0549
        12       -9090.2186          +0.0283
        13       -9090.2031          +0.0155
        14       -9090.1940          +0.0091


Training done. Model has been dump to  models/toi.pkl
==== nguoi


         1      -15723.4150             +nan
         2      -11954.4632       +3768.9519
         3      -11918.5697         +35.8935
         4      -11903.6349         +14.9348
         5      -11894.8700          +8.7649
         6      -11891.5776          +3.2925
         7      -11890.4077          +1.1699
         8      -11889.9474          +0.4602
         9      -11889.7351          +0.2123
        10      -11889.6025          +0.1326
        11      -11889.4998          +0.1027
        12      -11889.4166          +0.0832
        13      -11889.3539          +0.0626
        14      -11889.3121          +0.0419
        15      -11889.2869          +0.0252
        16      -11889.2728          +0.0141
        17      -11889.2652          +0.0076


Training done. Model has been dump to  models/nguoi.pkl
==== dich


...

         1      -12987.4135             +nan
         2       -9946.2311       +3041.1824
         3       -9902.0649         +44.1662
         4       -9875.2776         +26.7873
         5       -9870.4655          +4.8121
         6       -9869.9715          +0.4941
         7       -9869.6483          +0.3232
         8       -9869.3980          +0.2503
         9       -9869.1369          +0.2611
        10       -9868.6660          +0.4709
        11       -9867.8926          +0.7735
        12       -9867.2772          +0.6154
        13       -9867.0289          +0.2483
        14       -9866.9019          +0.1269
        15       -9866.7616          +0.1404
        16       -9866.5936          +0.1680
        17       -9866.4356          +0.1580
        18       -9866.3155          +0.1201
        19       -9866.2288          +0.0867
        20       -9866.1548          +0.0740
        21       -9866.0646          +0.0901
        22       -9865.8990          +0.1656
        23

Training done. Model has been dump to  models/theo.pkl
==== benh_nhan


         1      -28083.2378             +nan
         2      -24224.5508       +3858.6870
         3      -24174.1747         +50.3761
         4      -24165.4138          +8.7609
         5      -24161.5489          +3.8649
         6      -24158.6906          +2.8583
         7      -24156.4412          +2.2495
         8      -24155.0648          +1.3764
         9      -24154.3386          +0.7261
        10      -24153.9372          +0.4014
        11      -24153.6984          +0.2389
        12      -24153.5517          +0.1467
        13      -24153.4556          +0.0961
        14      -24153.3839          +0.0717
        15      -24153.3188          +0.0651
        16      -24153.2436          +0.0752
        17      -24153.1337          +0.1098
        18      -24152.9464          +0.1873
        19      -24152.6444          +0.3021
        20      -24152.2781          +0.3663
        21      -24151.9285          +0.3497
        22      -24151.5817          +0.3468
        23

Training done. Model has been dump to  models/benh_nhan.pkl
Training Done


        28      -24150.1637          +0.0011


In [29]:
#TEST
class_tests = ["test_toi", "test_nguoi", "test_theo", "test_dich", "test_benh_nhan"]
print("Testing and Labeling")
testset = {}
for name in class_tests:
  testset[name] = get_class_data(os.path.join("test", name))

  # Get all vectors in the datasets
all_train_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in testset.items()], axis=0)
print("vectors", all_train_vectors.shape)
  # Run K-Means algorithm to get clusters
kmeans = clustering(all_train_vectors)
print("centers", kmeans.cluster_centers_.shape)

for name in class_tests:
  testset[name] = list([kmeans.predict(v).reshape(-1,1) for v in testset[name]])

for true_cname in class_tests:
  if true_cname[:4] == "test":
    print("==================================")
    print(true_cname)
    print("==================================")

    lname = true_cname[5:]
    totalWord = 0
    true = 0
    accuracy = 0

    for O in testset[true_cname]:
      totalWord += 1
      scores = {}
      for cname, model in models.items():
        if cname[:4] != "test":
          score = model.score(O, [len(O)])
          scores[cname] = score
#     print(scores)
      srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
#     print(srt[0])

      if srt[0][0] == lname:
        true += 1
    accuracy = true/totalWord
    print("--------------------------------------------")
    print("!note: test_folder must contain wavs that it records exactly the word which be trained")
    print("accuracy: ", accuracy, ",true: ", true, ",total_word: ", totalWord)

Testing and Labeling


/home/apple/anaconda3/envs/voice/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


vectors (9175, 36)
centers (20, 36)
centers (20, 36)
test_toi
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.5643564356435643 ,true:  57 ,total_word:  101
test_nguoi
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.19480519480519481 ,true:  15 ,total_word:  77
test_theo
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.0 ,true:  0 ,total_word:  16
test_dich
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.8 ,true:  28 ,total_word:  35
test_benh_nhan
--------------------------------------------
!note: test_folder must contain wavs that it records exactly the word which be trained
accuracy:  0.0 ,true:  0 ,tot